## Segmentation

Image segmentation can be classified into three categories:
Semantic segmentation
Instance segmentation
Panoptic segmentation

The document will cover the following topics:
- Segmentation as pixel wise classification
    - Probabilistic classification
    - Mixture of Gaussians, EM

- Segmentation as energy minimization
    - Markov Random Fields
    - Energy formulation

- Graph cuts for image segmentaton
    - Basic idea
    - s-t Mincut Algorithm
    - Extension to non-binary case

\
The goal of segmentation is to identify the boundaries between different objects in an image. This topic is important because:
- it is a more basic step for the convolutional filters in Neural Networks for extracting image features
- it is the basis for many other image processing tasks such as object detection, object tracking, and image classification

\
Definition of The Problem\
Identifying groups of pixels in the input which is the image. This is a semantic segmentation, this means all the pixels that belong to a title are grouped together. For example, semantically meaningful groups such as color similarity,




Segmentation Approaches:
- Unsupervised Clustering
    - Grouping what "looks similar"
- Semantic Segmentation
    - Learn a classifier to assign a semantic class $C_k$ to every pixel




To define the grouping semantics, we use feature space. Grayscale image pixels are classified based on intensity similarity as a 1D representation, while colored images are classified based on color value similarity as a 3D representation.

A filter Bank of 24 filters allow us to operate in 24-dimension. 


\
Basically the method is to apply a threshold, above which is the foreground, and below is the background.

\
Probabilistic Classification
- Bayesian Classification
    - Given a measurement $x$, what semantic class $C_k$ should we assign to a pixel?
    - We must recall Bayes Decision Theory in this section:
        - $P(C_k|x) = \frac{p(x|C_k)p(C_k)}{\sum_{j}p(x|C_j)p(C_j)} $
        - where: 
            - $p(x|C_k)$ is the likelihood of the measurement $x$ having been generated by class $C_k$
            -  $p(C_k)$ is the prior probability of class $C_k$
    - In order to build a classifier, we can try either:
        - Discriminative Methods: directly estimating the posterior
        - Generative Methods: estimating likelihood and prior, and then using the Bayes Decision formula

\
Mixture Model of Normal Distribution (Gaussian) MoG \
Will be referred as MoG hereon


\
Expectation-Maximization (EM) Algorithm
 - E-Step: assign samples to mixture model components:
    - $ \pi_j \gamma_j(x_n) $


User Assisted Image Segmentation
- User marks two regions for foreground and background
- Learn a MoG model for the color values in each region
- Use the models to classify all pixels by deciding for the class with the highest posterior probability

### Pros of MoG, EM
- 

It is called generative model because values can be generate from the distribution

Cons
- Local minima
    - k-means is NP-hard (see: computational complexity theory, nondeterministic polynomial time) even with k=2
- Initialization
    - Often a good idea to start with some k-means iteration
- Needs to know number of componens
    - Solution: Model selection
- Needs careful implementation to avoid numerical issues


### References for this Section

---

[1] v7labs, https://www.v7labs.com/blog/panoptic-segmentation-guide

[2] fiveMinuteStats, EM, https://stephens999.github.io/fiveMinuteStats/intro_to_em.html

[3] fiveMinuteStates, Mixture Models, https://stephens999.github.io/fiveMinuteStats/intro_to_mixture_models.html

[4] neptune.ai, https://neptune.ai/blog/image-segmentation



&nbsp;

So far we have explored  bottom-up ways for segmentation, for which we examined individual pixels and neighborhoods to segment an image into regions, yet finding meaningful segments is intertwined with the recognition problem. 

In the next section, we will explore pixel neighborhood relations



### Segmentation: Caveats

Markov Random Fields (MRF) are a class of graphical models that are used to model the conditional probability of a random variable given its parents.

MRF Nodes as Pixels

![](Segmentation/img/mrf.png)




### Energy Formulation
Energy Function\
$E(x,y) = \sum_{i} + \sum_{i,j}\psi (x_i,x_j)$ \
$ -logp(x,y) = - \sum_{i} \log \psi(x_i,y_i)$

Local Optima of Energy Function

